In [3]:
import pandas as pd
import numpy as np
from datetime import date, timedelta, datetime

In [2]:
PV1_all = pd.read_csv('Data\enel_data\PV_data\PV1.csv')  #read the PV1 data file to get the solar generation data
PV1_filtered = PV1_all[PV1_all['Alarm'] == 0] # Filter out zeros 
PV1_filtered["V"]=PV1_filtered["V"].apply(pd.to_numeric) #ensure all voltage laues are numeric



PV1_filtered = PV1_filtered[PV1_filtered["V"] < 214748360]  #filter out misreadins as enel tells us to in their email
PV1_filtered = PV1_filtered[['timestamp','I_shunt','V']]
PV1_filtered["Power_kW"]= PV1_filtered['I_shunt']*PV1_filtered['V']/1000 #Power = I*V   (dived by 1000 to get units of kw)
PV1_filtered['timestamp'] = pd.to_datetime(PV1_filtered['timestamp'],errors = 'coerce') # change timestamps from strings to datetime objects so we can index and resample. takes like 5 min 
PV1_filtered.index=PV1_filtered["timestamp"] # set index to be our datetime so we can resample

PV1_resampled=PV1_filtered.resample('15T').mean() #resample power reading to the mean of every 15 minute interval

#need to isolate the timestamp from the year it was taken so we can merge with the timestamps of a different year for the house load data (since we do not have solar data available for every house)
PV1_resampled["month1"]=pd.DatetimeIndex(PV1_resampled.index).month.astype(str) #create month column
PV1_resampled["day1"]=pd.DatetimeIndex(PV1_resampled.index).day.astype(str) #create day column
PV1_resampled["time1"]=pd.DatetimeIndex(PV1_resampled.index).time.astype(str) # create time column



PV1_resampled['marker1'] = PV1_resampled[['month1', 'day1', "time1"]].agg('-'.join, axis=1) #create our mark column that we will use to merge this df with the house load df later
PV1_resampled=PV1_resampled.drop(columns=["I_shunt","V"]) #we no longer need these values

# PV2_all = pd.read_csv('Data\enel_data\PV_data\PV2.csv')
# PV2_filtered = PV2_all[PV2_all['Alarm'] == 0] # Filter out zeros 
# PV2_filtered["V"]=PV2_filtered["V"].apply(pd.to_numeric)


# PV2_filtered = PV2_filtered[PV2_filtered["V"] < 214748360]
# PV2_filtered = PV2_filtered[['timestamp','I_shunt','V']]
# PV2_filtered["Power_kW"]= PV2_filtered['I_shunt']*PV2_filtered['V']/1000

# PV2_filtered['timestamp'] = pd.to_datetime(PV2_filtered['timestamp'],errors = 'coerce') # takes like 5 min 
# PV2_filtered.index=PV2_filtered["timestamp"]

C:\Users\Caleb\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [3]:
# create list of files we want to sort through

files=["file_201907.csv","file_201906.csv","file_201905.csv","file_201904.csv","file_201903.csv","file_201902.csv","file_201901.csv","file_201812.csv","file_201811.csv","file_201810.csv","file_201809.csv","file_201808.csv"]
test=["file_201907.csv"]

In [4]:
#create a dataframe that will be appended with more house data from the following loops

cleaned_data=pd.DataFrame(columns=["dataid","local_15min","load","solar"])

# df.append(df2, ignore_index=True)

In [5]:
for file in files: #iterate through all files given above
    
    filepath= "Data\\enel_data\\NILM_data\\" + file  
                      
    all_data = pd.read_csv(filepath,delimiter = ';',encoding = 'utf8') #read this file
    
    serial_buffer=all_data[all_data['Serial_Number'] != "Serial_Number"][['Serial_Number','Data','Attiva']] #get rid of erroneous rows
    serial_buffer=serial_buffer[serial_buffer["Attiva"].map(type)== int]
    
    serial_buffer['Serial_Number']=pd.to_numeric(serial_buffer['Serial_Number']) #someserial numbers are given as strings so lets convert all to inttegers
    serial_buffer['Data'] = pd.to_datetime(serial_buffer['Data'],errors = 'coerce') # convert timestamp to datetime object.  takes like 5 min 
    house_ids=serial_buffer["Serial_Number"].unique() #get a list of all Serial Numbers (houses) in this file

    print(house_ids)
    
    for house_id in house_ids: #do a new interation for each separate home that we found in this file
        
        house= serial_buffer[serial_buffer['Serial_Number'] == house_id] #select data for this home
        house = house.rename(columns={'Data':'time','Attiva':'load'}) #load is in mW
        house.index = house['time'] #change index 
        house = house.drop(columns='time')
        house["load"] = house["load"].apply(pd.to_numeric, errors='coerce') #make all load values numeric
        house = house.resample('15T').mean() #resample house load to average load every 15 min

        house['load'] = house['load'] / 1000 / 1000 # convert mW to kW
        
        
        house["t_stamp"]=house.index #make new column of datetime we can break apart
        
        #same as PV data: make a marker we will use to merge the two df by taking the year out of the timestamp
        house["month2"]=pd.DatetimeIndex(house.index).month.astype(str)
        house["day2"]=pd.DatetimeIndex(house.index).day.astype(str)
        house["timey2"]=pd.DatetimeIndex(house.index).time.astype(str)

        house['marker2'] = house[['month2', 'day2', "timey2"]].agg('-'.join, axis=1) #create the second marker
        
        merged= pd.merge(house,PV1_resampled, how='inner', left_on="marker2", right_on="marker1") #merge the two df using the markers we made. data from the same realtive date and time interval will be combined
        df_append= merged.loc[:,["Serial_Number","t_stamp","load","Power_kW"]] #select the columns we want to keep
        df_append = df_append.rename(columns={'Serial_Number':'dataid','t_stamp':'local_15min','Power_kW':'solar'})
        
        cleaned_data=cleaned_data.append(df_append, ignore_index=True) #add the merged load and solar data for all homes in this file to the larger df we are creating

C:\Users\Caleb\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (0,1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


[118000000008 118000000012 118000000019]
[118000000008 118000000012 118000000019]
[118000000008 118000000012 118000000019]
[118000000008 118000000012 118000000019]
[118000000008 118000000012 118000000019]
[118000000008 118000000012 118000000019]
[118000000008 118000000012 118000000019]
[118000000008 118000000012]
[118000000008 118000000012]
[118000000008 118000000010 118000000012]
[118000000008 118000000010 118000000012]
[118000000008 118000000010 118000000012]


In [6]:
no_nans=cleaned_data[pd.notna(cleaned_data["dataid"])]
no_nans = no_nans.rename(columns={'local_15min':'time'})
no_nans["car1"] = 0
no_nans["dataid"]= no_nans["dataid"].apply(int)


In [7]:
no_nans

,dataid,time,load,solar,car1
0,118000000008,2019-01-07 00:00:00,0.077571,-0.002998,0
1,118000000008,2019-01-07 00:15:00,0.070403,-0.000574,0
2,118000000008,2019-01-07 00:30:00,0.079875,0.000000,0
3,118000000008,2019-01-07 00:45:00,0.066261,-0.001313,0
4,118000000008,2019-01-07 01:00:00,0.070168,-0.002968,0
...,...,...,...,...,...
663707,118000000012,2018-03-08 16:30:00,0.118551,0.000000,0
663708,118000000012,2018-03-08 16:45:00,0.131403,0.000000,0
663709,118000000012,2018-03-08 17:00:00,0.083446,0.000000,0
663710,118000000012,2018-03-08 17:15:00,0.056625,0.000000,0


In [4]:
enel_data=no_nans

# enel_data= pd.read_csv('cleaned_enel_data3.csv')


In [5]:
from datetime import datetime, timedelta

def datetime_range(start, end, delta):
    current = start
    while current < end:
        yield current
        current += delta


In [6]:

enel_data["time"]= pd.to_datetime(enel_data["time"]) # change to appropriate data type
enel_data["grid"]= enel_data["solar"]-enel_data["load"] #create grid value

new_enel = [dt.strftime('%Y-%m-%d %H:%M') for dt in 
       datetime_range(enel_data["time"].min(), enel_data["time"].max(), 
       timedelta(minutes=15))]

new_data=pd.DataFrame(columns=["time"])
new_data["time"]=pd.to_datetime(new_enel)

house_iter_ids=enel_data["dataid"].unique()


final_df=pd.DataFrame(columns=["dataid","local_15min","load","solar","grid"])

for house_iter_id in house_iter_ids:

    iter_data=enel_data.loc[enel_data["dataid"]==house_iter_id,:]
    iter_data=enel_data.merge(new_data, how='outer', left_on='time', right_on='time')
    iter_data["dataid"]=house_iter_id
    iter_data["car1"]=np.nan
    final_df=final_df.append(iter_data, ignore_index=True) 
    

    

C:\Users\Caleb\Anaconda3\lib\site-packages\pandas\core\frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


In [7]:
final_df

,car1,dataid,grid,load,local_15min,solar,time
0,NaN,118000000008,-0.080569,0.077571,NaN,-0.002998,2019-01-07 00:00:00
1,NaN,118000000008,-0.070977,0.070403,NaN,-0.000574,2019-01-07 00:15:00
2,NaN,118000000008,-0.079875,0.079875,NaN,0.000000,2019-01-07 00:30:00
3,NaN,118000000008,-0.067573,0.066261,NaN,-0.001313,2019-01-07 00:45:00
4,NaN,118000000008,-0.073136,0.070168,NaN,-0.002968,2019-01-07 01:00:00
...,...,...,...,...,...,...,...
342827,NaN,118000000010,NaN,NaN,NaN,NaN,2019-12-07 03:00:00
342828,NaN,118000000010,NaN,NaN,NaN,NaN,2019-12-07 03:15:00
342829,NaN,118000000010,NaN,NaN,NaN,NaN,2019-12-07 03:30:00
342830,NaN,118000000010,NaN,NaN,NaN,NaN,2019-12-07 03:45:00


In [8]:
final_df.to_csv('cleaned_enel_data4.csv', index = False)